In [1]:
import os
import pandas as pd
import nltk
import operator

In [2]:
#reading the files
fileList = []
path = 'Desktop/bbcsports/bbc_sports_articles'
for file in os.listdir(path):
    if os.path.isfile(os.path.join(path,file)):
        fileList.append(file)
fileList.reverse()

In [4]:
#checking to see if the files were accessed or not,
fileList

In [5]:
#it is time to follow the steps to build an inverted index:
# 1. first we fetch the documents
data = pd.DataFrame()
fileContent = []
documentId = []
i = 1
for f in fileList:
    content = open(os.path.join(path,f), "r")
    fileContent.append(content.read())
    
    documentId.append(i)
    i += 1
    content.close()

In [6]:
data['text'] = fileContent
data['DocId'] = documentId
data

,text,DocId
0,Dominici backs lacklustre France\n\nWing Chris...,1
1,Wales want rugby league training\n\nWales coul...,2
2,All Black magic: New Zealand rugby\n\nPlaying ...,3
3,Farrell saga to drag on - Lindsay\n\nWigan cha...,4
4,Disappointed Scott in solid start\n\nAllan Sco...,5
5,Collins calls for Chambers return\n\nWorld 100...,6
6,Wada will appeal against ruling\n\nThe World A...,7
7,Trial date is set for Balco case\n\nA US judge...,8
8,IAAF to rule on Greek sprint pair\n\nGreek spr...,9
9,South Africa recall spinner Boje\n\nSouth Afri...,10


In [8]:
#datastructure for clean articles
cleanArticles = []

for i in data['text']:
    cleanArticles.append(i.replace("\n", " ").replace("\'", " "))
    #getting rid of single character words
    cleanArticles = [" ".join([w for w in x.split() if len(w) > 1]) for x in cleanArticles]
    #converting everything to lowerCase to decrease the total number of unique
    #words
    cleanArticles = [x.lower() for x in cleanArticles]
    
data['PartlyCleanArticle'] = cleanArticles

In [18]:
#2. we now remove the stop words and punctuations
import string
punctuations = string.punctuation

from nltk.corpus import stopwords
stopword = stopwords.words("english")

def clean(text):
    cleanText = text
    #removing punctuations
    cleanText = "".join(word for word in cleanText if word not in punctuations)
    #time to remove the stop words
    words = cleanText.split(" ")
    words = [word for word in words if word not in stopword]
    cleanText = " ".join(words)
    return cleanText

In [19]:
data['clean'] = data['PartlyCleanArticle'].apply(clean)
data

,text,DocId,PartlyCleanArticle,clean
0,Dominici backs lacklustre France\n\nWing Chris...,1,dominici backs lacklustre france wing christop...,dominici backs lacklustre france wing christop...
1,Wales want rugby league training\n\nWales coul...,2,wales want rugby league training wales could f...,wales want rugby league training wales could f...
2,All Black magic: New Zealand rugby\n\nPlaying ...,3,all black magic: new zealand rugby playing col...,black magic new zealand rugby playing colours ...
3,Farrell saga to drag on - Lindsay\n\nWigan cha...,4,farrell saga to drag on lindsay wigan chairman...,farrell saga drag lindsay wigan chairman mauri...
4,Disappointed Scott in solid start\n\nAllan Sco...,5,disappointed scott in solid start allan scott ...,disappointed scott solid start allan scott con...
5,Collins calls for Chambers return\n\nWorld 100...,6,collins calls for chambers return world 100m c...,collins calls chambers return world 100m champ...
6,Wada will appeal against ruling\n\nThe World A...,7,wada will appeal against ruling the world anti...,wada appeal ruling world antidoping agency wad...
7,Trial date is set for Balco case\n\nA US judge...,8,trial date is set for balco case us judge has ...,trial date set balco case us judge set prelimi...
8,IAAF to rule on Greek sprint pair\n\nGreek spr...,9,iaaf to rule on greek sprint pair greek sprint...,iaaf rule greek sprint pair greek sprinters ko...
9,South Africa recall spinner Boje\n\nSouth Afri...,10,south africa recall spinner boje south africa ...,south africa recall spinner boje south africa ...


In [20]:
#3. it is now time to lemmatize the sentences
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

def lemmatizeText(text):
    words = text.split(" ")
    words = [lem.lemmatize(word, 'n') for word in words]
    words = [lem.lemmatize(word, 'v') for word in words]
    text = " ".join(words)
    return text

In [22]:
data['cleanLemmatizedText'] = data['clean'].apply(lemmatizeText)
data

,text,DocId,PartlyCleanArticle,clean,cleanLemmatizedText
0,Dominici backs lacklustre France\n\nWing Chris...,1,dominici backs lacklustre france wing christop...,dominici backs lacklustre france wing christop...,dominici back lacklustre france wing christoph...
1,Wales want rugby league training\n\nWales coul...,2,wales want rugby league training wales could f...,wales want rugby league training wales could f...,wale want rugby league train wale could follow...
2,All Black magic: New Zealand rugby\n\nPlaying ...,3,all black magic: new zealand rugby playing col...,black magic new zealand rugby playing colours ...,black magic new zealand rugby play colour blac...
3,Farrell saga to drag on - Lindsay\n\nWigan cha...,4,farrell saga to drag on lindsay wigan chairman...,farrell saga drag lindsay wigan chairman mauri...,farrell saga drag lindsay wigan chairman mauri...
4,Disappointed Scott in solid start\n\nAllan Sco...,5,disappointed scott in solid start allan scott ...,disappointed scott solid start allan scott con...,disappoint scott solid start allan scott confi...
5,Collins calls for Chambers return\n\nWorld 100...,6,collins calls for chambers return world 100m c...,collins calls chambers return world 100m champ...,collins call chamber return world 100m champio...
6,Wada will appeal against ruling\n\nThe World A...,7,wada will appeal against ruling the world anti...,wada appeal ruling world antidoping agency wad...,wada appeal rule world antidoping agency wada ...
7,Trial date is set for Balco case\n\nA US judge...,8,trial date is set for balco case us judge has ...,trial date set balco case us judge set prelimi...,trial date set balco case u judge set prelimin...
8,IAAF to rule on Greek sprint pair\n\nGreek spr...,9,iaaf to rule on greek sprint pair greek sprint...,iaaf rule greek sprint pair greek sprinters ko...,iaaf rule greek sprint pair greek sprinter kos...
9,South Africa recall spinner Boje\n\nSouth Afri...,10,south africa recall spinner boje south africa ...,south africa recall spinner boje south africa ...,south africa recall spinner boje south africa ...


In [23]:
def all_indices(value, qlist):
    indices = []
    idx = -1
    while True:
        try:
            idx = qlist.index(value, idx+1)
            indices.append(idx)
        except ValueError:
            break
    return indices

In [53]:
#Now that we have cleaned the text, removed stopwords, puntuations and lemmatized it, it is time to build a 
#inverted index (a very basic one)
inputData = data['cleanLemmatizedText']
invertedIndex = dict()
wordsChecked = set()
i = 1
for text in inputData:
    words = text.split(" ")
    for word in words:
        indices = all_indices(word, words)
        if word not in wordsChecked:
            value = {"id": i, 
                     "termFrequency" : len(indices) / len(words)}
            if word not in invertedIndex:
                temp = []
                invertedIndex[word] = temp
                invertedIndex[word].append(value)
            else:
                invertedIndex[word].append(value)
        wordsChecked.add(word)
    i += 1
    wordsChecked.clear()

In [54]:
invertedIndex

{'dominici': [{'id': 1, 'termFrequency': 0.027472527472527472},
  {'id': 36, 'termFrequency': 0.004405286343612335},
  {'id': 64, 'termFrequency': 0.004608294930875576},
  {'id': 81, 'termFrequency': 0.0043859649122807015},
  {'id': 195, 'termFrequency': 0.005988023952095809},
  {'id': 442, 'termFrequency': 0.0043859649122807015},
  {'id': 470, 'termFrequency': 0.0048543689320388345}],
 'back': [{'id': 1, 'termFrequency': 0.02197802197802198},
  {'id': 3, 'termFrequency': 0.0010235414534288639},
  {'id': 6, 'termFrequency': 0.01282051282051282},
  {'id': 9, 'termFrequency': 0.006666666666666667},
  {'id': 11, 'termFrequency': 0.014634146341463415},
  {'id': 12, 'termFrequency': 0.003484320557491289},
  {'id': 16, 'termFrequency': 0.002570694087403599},
  {'id': 18, 'termFrequency': 0.008064516129032258},
  {'id': 21, 'termFrequency': 0.011467889908256881},
  {'id': 22, 'termFrequency': 0.012345679012345678},
  {'id': 25, 'termFrequency': 0.0125},
  {'id': 26, 'termFrequency': 0.0071428

In [56]:
for key in invertedIndex:
    if type(invertedIndex[key])  is list:
        invertedIndex[key].sort(key=operator.itemgetter('termFrequency'), reverse = True)

In [57]:
#Check to see if sorting works as intended
invertedIndex['france']

[{'id': 114, 'termFrequency': 0.047058823529411764},
 {'id': 54, 'termFrequency': 0.046875},
 {'id': 195, 'termFrequency': 0.041916167664670656},
 {'id': 1, 'termFrequency': 0.038461538461538464},
 {'id': 232, 'termFrequency': 0.03389830508474576},
 {'id': 431, 'termFrequency': 0.03225806451612903},
 {'id': 36, 'termFrequency': 0.030837004405286344},
 {'id': 146, 'termFrequency': 0.024896265560165973},
 {'id': 470, 'termFrequency': 0.024271844660194174},
 {'id': 211, 'termFrequency': 0.02142857142857143},
 {'id': 356, 'termFrequency': 0.02109704641350211},
 {'id': 85, 'termFrequency': 0.0205761316872428},
 {'id': 407, 'termFrequency': 0.020202020202020204},
 {'id': 38, 'termFrequency': 0.01910828025477707},
 {'id': 81, 'termFrequency': 0.017543859649122806},
 {'id': 442, 'termFrequency': 0.017543859649122806},
 {'id': 448, 'termFrequency': 0.016286644951140065},
 {'id': 18, 'termFrequency': 0.016129032258064516},
 {'id': 272, 'termFrequency': 0.015306122448979591},
 {'id': 469, 'termFr

In [59]:
#final Inverted Index:
invertedIndex

{'dominici': [{'id': 1, 'termFrequency': 0.027472527472527472},
  {'id': 195, 'termFrequency': 0.005988023952095809},
  {'id': 470, 'termFrequency': 0.0048543689320388345},
  {'id': 64, 'termFrequency': 0.004608294930875576},
  {'id': 36, 'termFrequency': 0.004405286343612335},
  {'id': 81, 'termFrequency': 0.0043859649122807015},
  {'id': 442, 'termFrequency': 0.0043859649122807015}],
 'back': [{'id': 302, 'termFrequency': 0.03488372093023256},
  {'id': 29, 'termFrequency': 0.031331592689295036},
  {'id': 201, 'termFrequency': 0.02702702702702703},
  {'id': 464, 'termFrequency': 0.026785714285714284},
  {'id': 232, 'termFrequency': 0.022598870056497175},
  {'id': 1, 'termFrequency': 0.02197802197802198},
  {'id': 435, 'termFrequency': 0.02040816326530612},
  {'id': 399, 'termFrequency': 0.018691588785046728},
  {'id': 454, 'termFrequency': 0.017391304347826087},
  {'id': 220, 'termFrequency': 0.016666666666666666},
  {'id': 138, 'termFrequency': 0.01646090534979424},
  {'id': 100, 'te